<a href="https://colab.research.google.com/github/HarlanZhou/Harlanzhou.github.io/blob/master/GCN_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# loading package
import torch
import math
import torch.optim
import h5py
import numpy as np
import pandas as pd
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader,TensorDataset 
from torch import nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable

In [0]:
torch.set_default_tensor_type(torch.DoubleTensor)
# loading data

# loading pearson FC feature
feature = pd.read_excel('FC39.xlsx')

# loading adjacency matricx
# adjacency matricx shape: 223,116,116  samples*roinum*roinum
file = h5py.File('transformsAdj.mat','r')
# first loop means the num of dynamic FC
# second loop means the num of adjacency matricx
for i in range(1):
    data = [file[element[i]][:] for element in file['transforms']]
    data = np.array(data)

# loading label
label = pd.read_excel('label.xlsx')

# data transform
feature = np.array(feature)
label = np.array(label)

# numpy to Tensor
feature = torch.from_numpy(feature)
data = torch.from_numpy(data)
label = torch.from_numpy(label)

In [0]:
# construction Graph convolution
class GraphConvolution(nn.Module):
    def __init__(self,in_features,out_features,bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
      
    # init weight parameters
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
       
    # forward   
    def forward(self, input, adj):
        # support = XW  , input is feature , adj means adjacentMatricx
        support = torch.mm(input, self.weight)
        # out = A*support = AXW
        output = torch.spmm(adj, support)
        # add bias
        if self.bias is not None:
            return output + self.bias
        else:
            return output
    

# create GCN model 
class GCN(nn.Module):
    def __init__(self,nfeat,nhid,nclass,dropout):
        super(GCN, self).__init__() 
        self.gc1 = GraphConvolution(nfeat,nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout  # avoid overfitting
        
    def forward(self, x, adj): # x表示特征  adj表示邻接矩阵
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x,self.dropout, training=self.training)
        x = self.gc2(x,adj)
        return F.log_softmax(x,dim=1)

In [0]:
# other function
# override dataset
class MyDataset(Data.Dataset):
    def __init__(self, data1, data2, labels):
        self.data1 = data1
        self.data2 = data2
        self.labels = labels
    # override getitem, dataloader can get three parameters
    def __getitem__(self, index):
        img1, img2, target = self.data1[index], self.data2[index], self.labels[index]
        return img1,img2,target
    
    def __len__(self):
        return len(self.data1)

# implyment normalize matricx A
def Normalize(A):
    # add an Identity Matrix
    A = A + torch.eye(A.size(0))
    # if A is a symmetry matricx
    # d means get each node degree 
    d = A.sum(1)
    # D = D^-0.5
    D = torch.diag(torch.pow(d,-0.5))
    # return D^-0.5 * A * D^-0.5
    return D.mm(A).mm(D)


# Frobenius 函数
def Frobenius(A,B):
    return np.linalg.norm(A-B,ord='fro')

# 矩阵相似度函数 返回一个相似程度
def similarit1(A,B):
    differ = A - B
    dist = np.linalg.norm(differ, ord='fro')
    len1 = np.linalg.norm(A, ord='fro')
    len2 = np.linalg.norm(B, ord='fro')     # 普通模长
    denom = (len1 + len2) / 2
    similar = 1 - (dist / denom)
    return similar

# 2nd simiarity to measure
def similarity2(A,B):
    diff = A-B
    numera = torch.sum(diff**2)
    denom = torch.sum(A**2)
    similar = 1 - (numera/denom)
    return similar



# implyment accuracy
def accuracy(output,label,index_test):
    length = len(index_test)
    correct = torch.eq(output[index_test].argmax(dim=1),label[index_test].argmax(dim=1)).float().sum()
    acc = correct/length
    return acc

In [5]:
# cuda

# setting data
# combine feature and label together
# one hot format
label = torch.LongTensor(label)
label=torch.zeros(223,2).scatter_(1,label,1)

# construction adj matricx
# 采用similarity2方法，当返回的绝对值小于等于1认为邻接矩阵为1否则为0
# adapt similarity2, when returen value samller or equal than 1
# setting value 1, or setting value 0
adjmatricx = np.zeros([223,223])
for i in range(223):
    for j in range(i,223):
        if abs(similarity2(data[i],data[j]))<2 and i!=j:
            adjmatricx[i,j] = 1
            adjmatricx[j,i] = 1
        else:
            adjmatricx[i,j] = 0
            adjmatricx[j,i] = 0
            
# 自身不可达
adjmatricx = torch.from_numpy(adjmatricx)
adjmatricx = Normalize(adjmatricx)

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [0]:
# setting Hyperparameter
LR = 0.0003
BATCH_SIZE = 10
LM = 0.9
EPOCH = 200
DROPOUT = 0.5
WEIGHT_DECAY = 0.0001

NFEAT =6670
NHID = 16
NCLASS = 2
DROPOUT = 0.5

In [7]:
# split dataset into train set and test data
index_train,index_test = train_test_split(range(223),test_size=0.1)
print('train set the num is ',len(index_train))
print('test set the num is',len(index_test))
# 排序
index_train.sort()
index_test.sort()

# init Model and Parameters
gcn = GCN(NFEAT,NHID,NCLASS,DROPOUT)
gcn = gcn.double()

# optim and loss function
# optimizer = torch.optim.SGD(gcn.parameters(),lr=LR, momentum=LM)
optimizer = torch.optim.Adam(gcn.parameters(),lr=LR,betas=(0.9,0.99))
#loss_func = nn.CrossEntropyLoss()
#loss_func = nn.BCELoss()
loss_func = nn.MSELoss()



train set the num is  200
test set the num is 23


In [8]:
# training 
for epoch in range(20000):
    output = gcn(feature,adjmatricx)      # gcn predict output
    output = output.view(223,2)
    loss = loss_func(output[index_train],label[index_train])   # CrossEntory loss function
    # absent decay weight
    optimizer.zero_grad()           # clear gradients for this training step
    loss.backward()                 # backparargartion
    optimizer.step()  
    print(loss.item())
    
   

流式输出内容被截断，只能显示最后 5000 行内容。
1.5983885297487341
1.5884064478683657
1.5909829527425787
1.59094316259165
1.5884929871942
1.5967575017282931
1.5987968542680842
1.6089151016564585
1.5892185174219309
1.587914997352032
1.6076126025750088
1.5929525314161601
1.5897477526811254
1.59774260623473
1.5995393607360393
1.5947911170891518
1.605030013806167
1.5887255555520974
1.5945215813582791
1.5879972222430236
1.6029318473343137
1.5863365223079668
1.5917177093040613
1.5866308891878582
1.5907800734844961
1.595341130064524
1.5966384183067743
1.6160367146813532
1.5842152462904402
1.5856203847957357
1.586886955599538
1.6065438071065408
1.593116455797549
1.5932228598510167
1.5948853190265162
1.590451100830757
1.615607327075916
1.5860203613513084
1.5873411352493088
1.595318400042813
1.5876996016970346
1.6044259788701545
1.5940300014685602
1.598628860292991
1.6000597228488835
1.6011401428118643
1.60114800363382
1.5856332671386275
1.5855965421211722
1.590027062935689
1.5856499260809667
1.598099193791842
1.589

In [9]:
# testing
accuracy(output,label,index_test)

tensor(1., dtype=torch.float32)